In [5]:
import pyspark
import os

# Create SparkSession
sparkql = pyspark.sql.SparkSession.builder.master('local').getOrCreate()

# Create path and file variables
data_dir = './data'
coffee_data = 'coffee.csv'

# Create coffee PySpark DataFrame
coffee_df = sparkql.read.csv(os.path.join(data_dir, coffee_data), header=True)

# Show columns, schema, and df
print(coffee_df.columns)
print(coffee_df.schema)
coffee_df.show(4)

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Currency']
StructType([StructField('Date', StringType(), True), StructField('Open', StringType(), True), StructField('High', StringType(), True), StructField('Low', StringType(), True), StructField('Close', StringType(), True), StructField('Volume', StringType(), True), StructField('Currency', StringType(), True)])
+----------+------+-----+------+------+------+--------+
|      Date|  Open| High|   Low| Close|Volume|Currency|
+----------+------+-----+------+------+------+--------+
|2000-01-03|122.25|124.0| 116.1| 116.5|  6640|     USD|
|2000-01-04|116.25|120.5|115.75|116.25|  5492|     USD|
|2000-01-05| 115.0|121.0| 115.0| 118.6|  6165|     USD|
|2000-01-06| 119.0|121.4| 116.5|116.85|  5094|     USD|
+----------+------+-----+------+------+------+--------+
only showing top 4 rows



In [12]:
# Change the data types of columns
from pyspark.sql.functions import to_date
from pyspark.sql.types import FloatType
from pyspark.sql.types import IntegerType

# Cast Date column as DateType with to_date method
coffee_df = coffee_df.withColumn('Date', to_date(coffee_df['Date'], format='yyyy-MM-dd'))
# Make sure date is properly formatted
coffee_df.select('Date').show(4)

# Cast other columns as FloatType
coffee_df = coffee_df.withColumn('Open', coffee_df['Open'].cast(FloatType()))
coffee_df = coffee_df.withColumn('High', coffee_df['High'].cast(FloatType()))
coffee_df = coffee_df.withColumn('Low', coffee_df['Low'].cast(FloatType()))
coffee_df = coffee_df.withColumn('Close', coffee_df['Close'].cast(FloatType()))

coffee_df = coffee_df.withColumn('Volume', coffee_df['Volume'].cast(IntegerType()))

print(coffee_df.schema)

+----------+
|      Date|
+----------+
|2000-01-03|
|2000-01-04|
|2000-01-05|
|2000-01-06|
+----------+
only showing top 4 rows

StructType([StructField('Date', DateType(), True), StructField('Open', FloatType(), True), StructField('High', FloatType(), True), StructField('Low', FloatType(), True), StructField('Close', FloatType(), True), StructField('Volume', IntegerType(), True), StructField('Currency', StringType(), True)])


In [16]:
from pyspark.sql.functions import round

# Create High-Low and Open-Close difference Columns
coffee_df = coffee_df.withColumn('Open_Close_Diff', round(coffee_df.Open - coffee_df.Close, 2))
coffee_df = coffee_df.withColumn('High_Low_Diff', round(coffee_df.High - coffee_df.Low, 2))
coffee_df.show(4)

+----------+------+-----+------+------+------+--------+---------------+-------------+
|      Date|  Open| High|   Low| Close|Volume|Currency|Open_Close_Diff|High_Low_Diff|
+----------+------+-----+------+------+------+--------+---------------+-------------+
|2000-01-03|122.25|124.0| 116.1| 116.5|  6640|     USD|           5.75|          7.9|
|2000-01-04|116.25|120.5|115.75|116.25|  5492|     USD|            0.0|         4.75|
|2000-01-05| 115.0|121.0| 115.0| 118.6|  6165|     USD|           -3.6|          6.0|
|2000-01-06| 119.0|121.4| 116.5|116.85|  5094|     USD|           2.15|          4.9|
+----------+------+-----+------+------+------+--------+---------------+-------------+
only showing top 4 rows



In [41]:
from pyspark.sql.functions import lit, when

# Create Boolean column based on Volume values
coffee_df = coffee_df.withColumn('volume_filter_100', when(coffee_df.Volume >= 100, lit(True)).otherwise(lit(False)))

# Making sure column was properly created
coffee_df.filter(coffee_df.volume_filter_100 == True).show(5)
coffee_df.filter(coffee_df.volume_filter_100 == False).show(10)

+----------+------+------+------+------+------+--------+---------------+-------------+-----------------+
|      Date|  Open|  High|   Low| Close|Volume|Currency|Open_Close_Diff|High_Low_Diff|volume_filter_100|
+----------+------+------+------+------+------+--------+---------------+-------------+-----------------+
|2000-01-03|122.25| 124.0| 116.1| 116.5|  6640|     USD|           5.75|          7.9|             true|
|2000-01-04|116.25| 120.5|115.75|116.25|  5492|     USD|            0.0|         4.75|             true|
|2000-01-05| 115.0| 121.0| 115.0| 118.6|  6165|     USD|           -3.6|          6.0|             true|
|2000-01-06| 119.0| 121.4| 116.5|116.85|  5094|     USD|           2.15|          4.9|             true|
|2000-01-07|117.25|117.75| 113.8|114.15|  6855|     USD|            3.1|         3.95|             true|
+----------+------+------+------+------+------+--------+---------------+-------------+-----------------+
only showing top 5 rows

+----------+------+------+----

In [42]:
# Create Absolute Value column
from pyspark.sql.functions import abs

coffee_df = coffee_df.withColumn('Open_Close_Abs', abs(coffee_df.Open_Close_Diff))

# Show Absolute Value column
coffee_df.select('Open_Close_Abs').show(10)

+--------------+
|Open_Close_Abs|
+--------------+
|          5.75|
|           0.0|
|           3.6|
|          2.15|
|           3.1|
|          5.95|
|           2.3|
|          1.15|
|           0.7|
|           5.2|
+--------------+
only showing top 10 rows



In [62]:
import pandas as pd
from pyspark.sql import Window
from pyspark.sql.functions import pandas_udf

# Create udf
@pandas_udf('long')
def compute_net(c1: pd.Series,
                c2: pd.Series,
                c3: pd.Series, 
                c4: pd.Series, 
                c5: pd.Series) -> float:
  return ((c1 + c2 + c3 + c4) / 4) * c5

# Create Winow
w = Window \
    .partitionBy('Open', 'High', 'Close', 'Low') \
    .rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

# Create net_values column
coffee_df = coffee_df.withColumn('net_values', compute_net(coffee_df.Open, coffee_df.High, coffee_df.Low, coffee_df.Close, coffee_df.Volume).over(w))

# Make sure net_values column is there 
coffee_df.select('net_values').show(10)

+----------+
|net_values|
+----------+
|      1838|
|      6182|
|     11605|
|      3811|
|      1195|
|      7243|
|    241355|
|      3007|
|      2516|
|    182294|
+----------+
only showing top 10 rows



In [44]:
from pyspark.sql.functions import avg

# Create net_sales column
coffee_df = coffee_df.withColumn('net_sales', avg(coffee_df.Open, coffee_df.High, coffee_df.Low, coffee_df.Close * coffee_df.Volume))

coffee_df.select(coffee_df.net_sales).show(4)

TypeError: avg() takes 1 positional argument but 4 were given